In [20]:
import sys
print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, \
    outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
import mne
import os
import numpy as np
import pandas as pd
from ieeg.calc.reshape import make_data_same
from ieeg.calc.stats import time_perm_cluster
from ieeg.calc.mat import LabeledArray

from utils import *
import matplotlib.pyplot as plt

from pandas import read_csv
import scipy.stats as stats
import joblib

from scipy.ndimage import label

import json
import pickle

# rsa toolbox imports
from rsatoolbox.io.mne import read_epochs
from rsatoolbox.data.ops import merge_datasets
from rsatoolbox.rdm import calc_rdm_movie
from rsatoolbox.rdm.calc import _parse_input
from rsatoolbox.util.build_rdm import _build_rdms
from rsatoolbox.rdm import compare
from rsatoolbox.vis import show_rdm
from rsatoolbox.vis.timecourse import plot_timecourse

from os.path import join, expanduser, basename
import glob, json
import numpy, tqdm, mne, pandas
import rsatoolbox
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot

from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

['C:\\Users\\jz421\\Desktop\\GlobalLocal\\IEEG_Pipelines', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\python311.zip', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\DLLs', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg', '', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32\\lib', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\Pythonwin', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\Pythonwin', 'C:/Users/j

In [21]:
subjects = ['D0057','D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103']
# load in subjects electrodes to rois dict. If it doesn't already exist, make it and then load it.
filename = 'subjects_electrodestoROIs_dict.json'
subjects_electrodestoROIs_dict = make_or_load_subjects_electrodes_to_rois_dict(filename, subjects)

Attempting to load the subjects' electrodes-to-ROIs dictionary...
Loaded data from subjects_electrodestoROIs_dict.json
Dictionary loaded successfully. Ready to proceed!


In [22]:
stimulus_experiment_conditions = {
    "Stimulus/i25.0/s25.0": {
        "BIDS_events": "Stimulus/i25.0/s25.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Stimulus/i25.0/s75.0": {
        "BIDS_events": "Stimulus/i25.0/s75.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Stimulus/i75.0/s25.0": {
        "BIDS_events": "Stimulus/i75.0/s25.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Stimulus/i75.0/s75.0": {
        "BIDS_events": "Stimulus/i75.0/s75.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Stimulus/i25.0/r25.0": {
        "BIDS_events": "Stimulus/i25.0/r25.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Stimulus/i25.0/r75.0": {
        "BIDS_events": "Stimulus/i25.0/r75.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Stimulus/i75.0/r25.0": {
        "BIDS_events": "Stimulus/i75.0/r25.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Stimulus/i75.0/r75.0": {
        "BIDS_events": "Stimulus/i75.0/r75.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Stimulus/c25.0/s25.0": {
        "BIDS_events": "Stimulus/c25.0/s25.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Stimulus/c25.0/s75.0": {
        "BIDS_events": "Stimulus/c25.0/s75.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Stimulus/c75.0/s25.0": {
        "BIDS_events": "Stimulus/c75.0/s25.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Stimulus/c75.0/s75.0": {
        "BIDS_events": "Stimulus/c75.0/s75.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Stimulus/c25.0/r25.0": {
        "BIDS_events": "Stimulus/c25.0/r25.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Stimulus/c25.0/r75.0": {
        "BIDS_events": "Stimulus/c25.0/r75.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Stimulus/c75.0/r25.0": {
        "BIDS_events": "Stimulus/c75.0/r25.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Stimulus/c75.0/r75.0": {
        "BIDS_events": "Stimulus/c75.0/r75.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "75%"
    }
}

# congruency_conditions = {
#     "Stimulus_c": {
#         "BIDS_events": ["Stimulus/c25/s25", "Stimulus/c25/s75", "Stimulus/c75/s25", "Stimulus/c75/s75", "Stimulus/c25/r25", "Stimulus/c25/r75", "Stimulus/c75/r25", "Stimulus/c75/r75"],
#         "congruency": "c"
#     }
# }

stimulus_conditions = {
    "Stimulus/BigLetters/SmallLetterh/Taskg": {
        "BIDS_events": "Stimulus/BigLetters/SmallLetterh/Taskg",
        "bigLetter": "s",
        "smallLetter": "h",
        "task": "g"
    },
    "Stimulus/BigLetters/SmallLetterh/Taskl": {
        "BIDS_events": "Stimulus/BigLetters/SmallLetterh/Taskl",
        "bigLetter": "s",
        "smallLetter": "h",
        "task": "l"
    },
    "Stimulus/BigLetters/SmallLetters/Taskg": {
        "BIDS_events": "Stimulus/BigLetters/SmallLetters/Taskg",
        "bigLetter": "s",
        "smallLetter": "s",
        "task": "g"
    },
    "Stimulus/BigLetters/SmallLetters/Taskl": {
        "BIDS_events": "Stimulus/BigLetters/SmallLetters/Taskl",
        "bigLetter": "s",
        "smallLetter": "s",
        "task": "l"
    },
    "Stimulus/BigLetterh/SmallLetterh/Taskg": {
        "BIDS_events": "Stimulus/BigLetterh/SmallLetterh/Taskg",
        "bigLetter": "h",
        "smallLetter": "h",
        "task": "g"
    },
    "Stimulus/BigLetterh/SmallLetterh/Taskl": {
        "BIDS_events": "Stimulus/BigLetterh/SmallLetterh/Taskl",
        "bigLetter": "h",
        "smallLetter": "h",
        "task": "l"
    },
    "Stimulus/BigLetterh/SmallLetters/Taskg": {
        "BIDS_events": "Stimulus/BigLetterh/SmallLetters/Taskg",
        "bigLetter": "h",
        "smallLetter": "s",
        "task": "g"
    },
    "Stimulus/BigLetterh/SmallLetters/Taskl": {
        "BIDS_events": "Stimulus/BigLetterh/SmallLetters/Taskl",
        "bigLetter": "h",
        "smallLetter": "s",
        "task": "l"
    }
}

response_experiment_conditions = {
    "Response/i25.0/s25.0": {
        "BIDS_events": "Response/i25.0/s25.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Response/i25.0/s75.0": {
        "BIDS_events": "Response/i25.0/s75.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Response/i75.0/s25.0": {
        "BIDS_events": "Response/i75.0/s25.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Response/i75.0/s75.0": {
        "BIDS_events": "Response/i75.0/s75.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Response/i25.0/r25.0": {
        "BIDS_events": "Response/i25.0/r25.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Response/i25.0/r75.0": {
        "BIDS_events": "Response/i25.0/r75.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Response/i75.0/r25.0": {
        "BIDS_events": "Response/i75.0/r25.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Response/i75.0/r75.0": {
        "BIDS_events": "Response/i75.0/r75.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Response/c25.0/s25.0": {
        "BIDS_events": "Response/c25.0/s25.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Response/c25.0/s75.0": {
        "BIDS_events": "Response/c25.0/s75.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Response/c75.0/s25.0": {
        "BIDS_events": "Response/c75.0/s25.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Response/c75.0/s75.0": {
        "BIDS_events": "Response/c75.0/s75.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Response/c25.0/r25.0": {
        "BIDS_events": "Response/c25.0/r25.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Response/c25.0/r75.0": {
        "BIDS_events": "Response/c25.0/r75.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Response/c75.0/r25.0": {
        "BIDS_events": "Response/c75.0/r25.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Response/c75.0/r75.0": {
        "BIDS_events": "Response/c75.0/r75.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "75%"
    }
}

response_conditions = {
    "Response/BigLetters/SmallLetterh/Taskg": {
        "BIDS_events": "Response/BigLetters/SmallLetterh/Taskg",
        "bigLetter": "s",
        "smallLetter": "h",
        "task": "g"
    },
    "Response/BigLetters/SmallLetterh/Taskl": {
        "BIDS_events": "Response/BigLetters/SmallLetterh/Taskl",
        "bigLetter": "s",
        "smallLetter": "h",
        "task": "l"
    },
    "Response/BigLetters/SmallLetters/Taskg": {
        "BIDS_events": "Response/BigLetters/SmallLetters/Taskg",
        "bigLetter": "s",
        "smallLetter": "s",
        "task": "g"
    },
    "Response/BigLetters/SmallLetters/Taskl": {
        "BIDS_events": "Response/BigLetters/SmallLetters/Taskl",
        "bigLetter": "s",
        "smallLetter": "s",
        "task": "l"
    },
    "Response/BigLetterh/SmallLetterh/Taskg": {
        "BIDS_events": "Response/BigLetterh/SmallLetterh/Taskg",
        "bigLetter": "h",
        "smallLetter": "h",
        "task": "g"
    },
    "Response/BigLetterh/SmallLetterh/Taskl": {
        "BIDS_events": "Response/BigLetterh/SmallLetterh/Taskl",
        "bigLetter": "h",
        "smallLetter": "h",
        "task": "l"
    },
    "Response/BigLetterh/SmallLetters/Taskg": {
        "BIDS_events": "Response/BigLetterh/SmallLetters/Taskg",
        "bigLetter": "h",
        "smallLetter": "s",
        "task": "g"
    },
    "Response/BigLetterh/SmallLetters/Taskl": {
        "BIDS_events": "Response/BigLetterh/SmallLetters/Taskl",
        "bigLetter": "h",
        "smallLetter": "s",
        "task": "l"
    }
}

In [23]:
task='GlobalLocal'

conditions = stimulus_conditions # toggle

if conditions == stimulus_conditions or stimulus_experiment_conditions:
    epochs_root_file = "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8"
elif conditions == response_conditions or response_experiment_conditions:
    epochs_root_file = "Response_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8"
else:
    raise ValueError("Unknown condition type.")

condition_names = list(conditions.keys()) # get the condition names as a list

# This breaks if just_HG_ev1_rescaled is set to False currently. Fix this! 8/11
subjects_mne_objects = create_subjects_mne_objects_dict(subjects=subjects, epochs_root_file=epochs_root_file, conditions=conditions, task="GlobalLocal", just_HG_ev1_rescaled=True, acc_trials_only=True)

Loading data for subject: D0057
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_HG_ev1_rescaled-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...    1500.00 ms
        0 CTF compensation matrices available
Not setting metadata
449 matching events found
No baseline correction applied
0 projection items activated
  Loading condition: Stimulus/BigLetters/SmallLetterh/Taskg with parameters: {'BIDS_events': 'Stimulus/BigLetters/SmallLetterh/Taskg', 'bigLetter': 's', 'smallLetter': 'h', 'task': 'g'}
  Loading condition: Stimulus/BigLetters/SmallLetterh/Taskl with parameters: {'BIDS_events': 'Stimulus/BigLetters/SmallLetterh/Taskl', 'bigLetter': 's', 'smallLetter': 'h', 'task': 'l'}
  Loading condition: Stimulus/BigLetters/SmallLetters/Taskg with parameters: {'BIDS_events': 'Stimulus/BigLetters/SmallLetters/Taskg', 'bigLetter': 's', 'smallLetter': 's', 'ta

In [24]:
condition_names = list(conditions.keys()) # get the condition names as a list
condition_names_bids = [condition['BIDS_events'] for condition in conditions.values()] # get the condition names in bids format
condition_names

['Stimulus/BigLetters/SmallLetterh/Taskg',
 'Stimulus/BigLetters/SmallLetterh/Taskl',
 'Stimulus/BigLetters/SmallLetters/Taskg',
 'Stimulus/BigLetters/SmallLetters/Taskl',
 'Stimulus/BigLetterh/SmallLetterh/Taskg',
 'Stimulus/BigLetterh/SmallLetterh/Taskl',
 'Stimulus/BigLetterh/SmallLetters/Taskg',
 'Stimulus/BigLetterh/SmallLetters/Taskl']

get significant channels

In [25]:
sig_chans_per_subject = get_sig_chans_per_subject(subjects, epochs_root_file, task='GlobalLocal', LAB_root=None)

Loaded significant channels for subject D0057
Loaded significant channels for subject D0059
Loaded significant channels for subject D0063
Loaded significant channels for subject D0065
Loaded significant channels for subject D0069
Loaded significant channels for subject D0071
Loaded significant channels for subject D0077
Loaded significant channels for subject D0090
Loaded significant channels for subject D0094
Loaded significant channels for subject D0100
Loaded significant channels for subject D0102
Loaded significant channels for subject D0103


In [26]:
# Assuming you have a list of subjects
root_dir = rf"C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs"


In [27]:
# Define your ROIs dictionary and other parameters
rois_dict = {
    'lpfc': ["G_front_inf-Opercular", "G_front_inf-Orbital", "G_front_inf-Triangul", "G_front_middle", "G_front_sup", "Lat_Fis-ant-Horizont", "Lat_Fis-ant-Vertical", "S_circular_insula_ant", "S_circular_insula_sup", "S_front_inf", "S_front_middle", "S_front_sup"],
    'v1': ["G_oc-temp_med-Lingual", "S_calcarine", "G_cuneus"],
    'occ': ["G_cuneus", "G_and_S_occipital_inf", "G_occipital_middle", "G_occipital_sup", "G_oc-temp_lat-fusifor", "G_oc-temp_med-Lingual", "Pole_occipital", "S_calcarine", "S_oc_middle_and_Lunatus", "S_oc_sup_and_transversal", "S_occipital_ant"],
    'occ_filtered': [],
    'occ_best_filtered': []
}

rois = list(rois_dict.keys())
# Assuming you have subjects_electrodestoROIs_dict and sig_chans_per_subject dictionaries
electrodes_per_subject_roi, sig_electrodes_per_subject_roi, = make_sig_electrodes_per_subject_and_roi_dict(
    rois_dict, subjects_electrodestoROIs_dict, sig_chans_per_subject
)

For subject D0057, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['RAI6', 'RAI12', 'RAI13', 'RAI15', 'RAI16', 'RPI14', 'RAMF10', 'RAMF11', 'RAMF12', 'RAIF11', 'RAIF12', 'RAIF13', 'RAIF14']
For subject D0059, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['LMMF9', 'LMMF11', 'LMMF12', 'LPSF16']
For subject D0063, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['LOF16', 'LASF10', 'LASF14', 'LASF15', 'LASF16', 'LM

let's manually make the occ filtered sig electrodes

In [28]:
sig_electrodes_per_subject_roi['occ_filtered'] = {
    'D0057': [],
    'D0059': [],
    'D0071': ['RO1', 'RO10'], #RO10 is iffy, big drop from fix onset
    'D0077': ['ROPM1', 'ROPM8'],
    'D0090': ['RTPO1', 'RTPI1'],
    'D0100': ['LOAI12', 'LOAI13', 'LOAI14', 'LOAI15', 'LOMI9', 'LOMI11', 'LOPI8', 'LOPI9', 'LOPM8', 'LOPM9', 'LPPI7', 'LPPI8', 'LPPI9'],
    'D0102': ['RTPI1'],
    'D0103': ['LTPI2', 'LTPI3', 'LTPI4']
}

and now make the best of the best

In [29]:
sig_electrodes_per_subject_roi['occ_best_filtered'] = {
    'D0057': [],
    'D0059': [],
    'D0071': [],
    'D0077': [],
    'D0090': [],
    'D0100': ['LOAI12', 'LOMI9', 'LOPI8', 'LOPI9', 'LOPM8', 'LOPM9'],
    'D0102': ['RTPI1'],
    'D0103': ['LTPI2', 'LTPI3', 'LTPI4']
}

In [30]:
# Specify the file path where you want to save the dictionary
save_path = 'sig_electrodes_per_subject_roi.json'

# Use json to save the dictionary
with open(save_path, 'w') as file:
    json.dump(sig_electrodes_per_subject_roi, file, indent=4)

print(f"Dictionary saved to {save_path}")

Dictionary saved to sig_electrodes_per_subject_roi.json


get electrode counts for each roi

In [31]:
total_electrodes_info = calculate_total_electrodes(sig_electrodes_per_subject_roi, electrodes_per_subject_roi)
for roi, counts in total_electrodes_info.items():
    print(f"Total number of {roi} electrodes across all subjects:", counts['total_electrodes'])
    print(f"Total number of significant {roi} electrodes across all subjects:", counts['total_significant_electrodes'])


Total number of lpfc electrodes across all subjects: 105
Total number of significant lpfc electrodes across all subjects: 46
Total number of v1 electrodes across all subjects: 17
Total number of significant v1 electrodes across all subjects: 12
Total number of occ electrodes across all subjects: 57
Total number of significant occ electrodes across all subjects: 41
Total number of occ_filtered electrodes across all subjects: 0
Total number of significant occ_filtered electrodes across all subjects: 23
Total number of occ_best_filtered electrodes across all subjects: 0
Total number of significant occ_best_filtered electrodes across all subjects: 10


check if any subjects have a weird sampling rate

In [32]:
# Assuming 'subjects_mne_objects' is your dictionary containing MNE objects for each subject
sampling_rate = 256
subject_rates = check_sampling_rates(subjects_mne_objects, expected_sampling_rate=sampling_rate)


Subject D0057 has the expected sampling rate: 256.0 Hz.
Subject D0059 has the expected sampling rate: 256.0 Hz.
Subject D0063 has the expected sampling rate: 256.0 Hz.
Subject D0065 has the expected sampling rate: 256.0 Hz.
Subject D0069 has the expected sampling rate: 256.0 Hz.
Subject D0071 has the expected sampling rate: 256.0 Hz.
Subject D0077 has the expected sampling rate: 256.0 Hz.
Subject D0090 has the expected sampling rate: 256.0 Hz.
Subject D0094 has the expected sampling rate: 256.0 Hz.
Subject D0100 has the expected sampling rate: 256.0 Hz.
Subject D0102 has the expected sampling rate: 256.0 Hz.
Subject D0103 has the expected sampling rate: 256.0 Hz.


create a dat dict like in the Temporal example from rsatoolbox 7/19  
#### Data Dictionary (dat) Structure
- **roi1**
  - **data**: 3D data array for this ROI
  - **channel_names**: List of significant channels for this ROI
  - **cond_names**: Dictionary where keys are condition names and values are integer indices
  - **cond_idx**: 1D array where each entry is an integer index corresponding to a cond_name
  - **times**: 1D array of times, corresponding to each sample
- **roi2**
  - **data**: 3D data array for this ROI
  - **channel_names**: List of significant channels for this ROI
  - **cond_names**: Dictionary where keys are condition names and values are integer indices
  - **cond_idx**: 1D array where each entry is an integer index corresponding to a cond_name
  - **times**: 1D array of times, corresponding to each sample
- **roiX**
  - **data**: 3D data array for this ROI
  - **channel_names**: List of significant channels for this ROI
  - **cond_names**: Dictionary where keys are condition names and values are integer indices
  - **cond_idx**: 1D array where each entry is an integer index corresponding to a cond_name
  - **times**: 1D array of times, corresponding to each sample


        




In [38]:
# Example call to the function (you need to replace the arguments with actual data)
dat = prepare_data_for_temporal_dataset(subjects_mne_objects, condition_names, rois, subjects, sig_electrodes_per_subject_roi)

subjects:  ['D0057', 'D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103']
num channels:  46
Processing D0057 for Stimulus/BigLetters/SmallLetterh/Taskg in lpfc
Number of trials for D0057 in Stimulus/BigLetters/SmallLetterh/Taskg: 59
sub:  D0057
channel indices:  [0, 1]
Processing D0057 for Stimulus/BigLetters/SmallLetterh/Taskl in lpfc
Number of trials for D0057 in Stimulus/BigLetters/SmallLetterh/Taskl: 54
sub:  D0057
channel indices:  [0, 1]
Processing D0057 for Stimulus/BigLetters/SmallLetters/Taskg in lpfc
Number of trials for D0057 in Stimulus/BigLetters/SmallLetters/Taskg: 53
sub:  D0057
channel indices:  [0, 1]
Processing D0057 for Stimulus/BigLetters/SmallLetters/Taskl in lpfc
Number of trials for D0057 in Stimulus/BigLetters/SmallLetters/Taskl: 34
sub:  D0057
channel indices:  [0, 1]
Processing D0057 for Stimulus/BigLetterh/SmallLetterh/Taskg in lpfc
Number of trials for D0057 in Stimulus/BigLetterh/SmallLetterh/Taskg: 51
sub:  D00

In [43]:
dat['lpfc']['data']

array([[[-0.22146894, -0.36678028, -0.45204559, ..., -1.13555562,
         -1.26031387, -1.36502028],
        [ 0.44666699,  0.45215368,  0.42189771, ..., -0.27700126,
         -0.25733578, -0.22684783],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan]],

       [[-0.23705637, -0.05304538,  0.15468368, ...,  1.32827187,
          1.02210057,  0.71134883],
        [-0.77331793, -0.94910252, -1.03917658, ...,  0.35906565,
          0.0732761 , -0.24011691],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [        nan,         nan,         nan, ...,  

try to use aaron's LabeledArray class to hold my data and conditions instead. Ugh this is broken. Trying to make 5D array of ROI x condition x trial x channel x timepoint. But annoying. 8/30.

In [41]:
# def put_data_in_labeled_array(subjects_mne_objects, condition_names, rois, subjects, sig_electrodes_per_subject_roi):
#     # Dictionary to hold the LabeledArray for each ROI
#     dat = {}

#     # Loop through each ROI
#     for roi in rois:
#         all_data = []
#         roi_labels = []
#         condition_labels = []
#         trial_labels = []
#         channel_labels = []
#         time_points = None

#         for cond_idx, condition_name in enumerate(condition_names):
#             condition_data = []
#             condition_trials = []
#             condition_channels = []

#             for sub in subjects:
#                 sig_electrodes = sig_electrodes_per_subject_roi[roi].get(sub, [])
#                 if not sig_electrodes:
#                     continue

#                 sub_channel_names = [sub + '-' + sig_electrode for sig_electrode in sig_electrodes]
#                 epochs = subjects_mne_objects[sub][condition_name]['HG_ev1_rescaled'].copy().pick(sig_electrodes)
#                 epochs_data = epochs.get_data(copy=True)

#                 if time_points is None:
#                     time_points = epochs.times

#                 num_trials, num_channels, _ = epochs_data.shape

#                 # Append the data for this condition
#                 condition_data.append(epochs_data)
#                 condition_trials.extend([f"trial_{i}" for i in range(num_trials)])
#                 condition_channels.extend(sub_channel_names)

#             if condition_data:
#                 all_data.append(np.concatenate(condition_data, axis=0))
#                 condition_labels.extend([condition_name] * len(condition_data))

#             trial_labels.extend(condition_trials)
#             channel_labels.extend(condition_channels)

#         if all_data:
#             # Concatenate data across all conditions for this ROI
#             roi_data = np.concatenate(all_data, axis=0)

#             # Creating labels
#             roi_labels = [roi] * len(all_data)
#             roi_labels = [roi for _ in range(len(condition_names))]  # Each condition is tied to the same ROI
#             labels = [
#                 roi_labels,           # ROI labels
#                 condition_labels,     # Condition labels
#                 trial_labels,         # Trial labels
#                 channel_labels,       # Channel labels
#                 time_points           # Time points
#             ]

#             # Store in dat dictionary
#             dat[roi] = LabeledArray(roi_data, labels=labels)

#     return dat

# # Example call to the function
# dat_labeled_array = put_data_in_labeled_array(subjects_mne_objects, condition_names, rois, subjects, sig_electrodes_per_subject_roi)


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 2 and the array at index 1 has size 4

attempt to do time resolved decoding 8/21

In [50]:
def create_time_bins(time_points, bin_size=0.1):
    """
    Create time bins from the given time points array.

    Parameters:
    - time_points: 1D numpy array of time points.
    - bin_size: The size of each time bin in seconds (default is 0.1s, which is 100 ms).

    Returns:
    - bins: A list of numpy arrays, where each array contains the time points within that bin.
    """
    bins = []
    start_time = np.min(time_points)
    end_time = np.max(time_points)
    current_time = start_time
    
    while current_time < end_time:
        next_time = current_time + bin_size
        if next_time > end_time:
            next_time = end_time
        bins.append(time_points[(time_points >= current_time) & (time_points < next_time)])
        current_time = next_time

    # Ensure that the last bin includes the endpoint
    if time_points[-1] not in bins[-1]:
        bins[-1] = np.append(bins[-1], time_points[-1])

    return bins

In [51]:
# Assuming temporal_datasets is a dictionary
first_roi = next(iter(dat.keys()))
time_points = dat[first_roi]['times']

# Create time bins
bins = create_time_bins(time_points, bin_size=0.1)
bins

[array([-1.        , -0.99609375, -0.9921875 , -0.98828125, -0.984375  ,
        -0.98046875, -0.9765625 , -0.97265625, -0.96875   , -0.96484375,
        -0.9609375 , -0.95703125, -0.953125  , -0.94921875, -0.9453125 ,
        -0.94140625, -0.9375    , -0.93359375, -0.9296875 , -0.92578125,
        -0.921875  , -0.91796875, -0.9140625 , -0.91015625, -0.90625   ,
        -0.90234375]),
 array([-0.8984375 , -0.89453125, -0.890625  , -0.88671875, -0.8828125 ,
        -0.87890625, -0.875     , -0.87109375, -0.8671875 , -0.86328125,
        -0.859375  , -0.85546875, -0.8515625 , -0.84765625, -0.84375   ,
        -0.83984375, -0.8359375 , -0.83203125, -0.828125  , -0.82421875,
        -0.8203125 , -0.81640625, -0.8125    , -0.80859375, -0.8046875 ,
        -0.80078125]),
 array([-0.796875  , -0.79296875, -0.7890625 , -0.78515625, -0.78125   ,
        -0.77734375, -0.7734375 , -0.76953125, -0.765625  , -0.76171875,
        -0.7578125 , -0.75390625, -0.75      , -0.74609375, -0.7421875 ,
     

first remove trials with nans, similar to how aaron does it in his nan_common_denom function in aaron_grouping.py

In [73]:
# isn = np.isnan(dat['lpfc']['data'])  # Assuming `dat[roi]['data']` is your data array
# nan_trials = np.any(isn, axis=(1, 2))  # This checks for NaNs across channels and time points
# nan_trials

array([ True,  True,  True, ...,  True,  True,  True])

In [61]:
# min_trials = 0
# for roi in rois:
#     # identify nans
#     isn = np.isnan(dat[roi]['data'])  # Assuming `dat[roi]['data']` is your data array
#     nan_trials = np.any(isn, axis=(1, 2))  # This checks for NaNs across channels and time points

#     # filter or remove trials with nans
#     valid_trials = ~nan_trials
#     dat[roi]['data'] = dat[roi]['data'][valid_trials]
#     dat[roi]['cond_idx'] = dat[roi]['cond_idx'][valid_trials]
#     dat[roi]['sub_idx'] = dat[roi]['sub_idx'][valid_trials]

#     # exclude channels with too few valid trials
#     ch_tnum = dat[roi]['data'].shape[0] - np.sum(nan_trials, axis=0)
#     if np.min(ch_tnum) < min_trials:
#         # Exclude channels with insufficient valid trials
#         valid_channels = ch_tnum >= min_trials
#         dat[roi]['data'] = dat[roi]['data'][:, valid_channels, :]

#     # restructure the data array after filtering
#     dat[roi]['data'] = dat[roi]['data'].reshape(-1, dat[roi]['data'].shape[1], dat[roi]['data'].shape[2])


IndexError: boolean index did not match indexed array along dimension 1; dimension is 0 but corresponding boolean dimension is 641

now actually run the decoding

In [58]:
import numpy as np
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC  # Example: Support Vector Classifier

def decode_conditions_time_binning(dat, conditions, classifier='logistic_regression', cv_folds=5, bin_size=0.1, classify_by='bigLetter', rois=None):
    decoding_results = {}

    # Choose the classifier
    if classifier == 'logistic_regression':
        clf = make_pipeline(LogisticRegression(max_iter=1000))
    elif classifier == 'hist_gradient_boosting':
        clf = make_pipeline(HistGradientBoostingClassifier())
    elif classifier == 'svm':
        clf = make_pipeline(SVC(kernel='linear'))  # Example: Linear SVM
    else:
        raise ValueError(f"Unsupported classifier: {classifier}")

    # Determine which ROIs to run the decoding on
    if rois is None:
        rois = dat.keys()  # Default to all ROIs in dat
    else:
        rois = [roi for roi in rois if roi in dat]

    # Extract time points from the first ROI in the `dat`
    first_roi = next(iter(dat.keys()))
    time_points = dat[first_roi].labels[-1]  # Assuming the last label is time_points

    # Create time bins
    bins = create_time_bins(time_points, bin_size=bin_size)

    # Create a map from condition names to the chosen attribute (e.g., bigLetter, smallLetter, or task)
    condition_map = {name: conditions[name][classify_by] for name in conditions}

    # Get unique labels for classification
    unique_labels = list(set(condition_map.values()))
    label_to_index = {label: i for i, label in enumerate(unique_labels)}

    # Loop over the selected ROIs in the data dictionary
    for roi in rois:
        roi_data = dat[roi]
        print(f'Decoding conditions for ROI: {roi}')

        # Extract data and original condition labels
        data = roi_data  # Since roi_data is already a LabeledArray
        cond_idx = data.labels[1]  # Assuming the second label dimension is condition indices
        sub_idx = data.labels[0]  # Assuming the first label dimension is subject indices

        # Create group labels based on the chosen attribute
        group_idx = np.array([label_to_index[condition_map[cond_name]] for cond_name in conditions.keys()])[cond_idx]

        # Calculate window indices based on the bins
        time_window_results = []
        num_trials, num_channels, _ = data.shape

        for time_bin in bins:
            bin_indices = np.isin(time_points, time_bin)
            data_window = data[:, :, bin_indices]  # Select data within the time bin

            # Identify and remove all-NaN channels
            nan_channels = np.all(np.isnan(data_window), axis=(0, 2))  # Find channels that are all NaNs
            data_window_valid_channels = data_window[:, ~nan_channels, :]  # Keep only non-NaN channels

            # Flatten the data for decoding
            num_valid_channels = data_window_valid_channels.shape[1]
            data_window_flat = data_window_valid_channels.reshape(num_trials, num_valid_channels * len(time_bin))

            # Perform cross-validated decoding between the two groups
            cv = StratifiedKFold(n_splits=cv_folds)
            scores = cross_val_score(clf, data_window_flat, group_idx, cv=cv)

            # Store the decoding accuracy for this time bin
            mean_accuracy = np.mean(scores)
            std_accuracy = np.std(scores)
            time_window_results.append({
                'time_window': (time_bin[0], time_bin[-1]),
                'mean_accuracy': mean_accuracy,
                'std_accuracy': std_accuracy
            })

            print(f"Time bin {time_bin[0]}s to {time_bin[-1]}s: Mean accuracy = {mean_accuracy:.4f}")

        # Store results for each ROI
        decoding_results[roi] = time_window_results

    return decoding_results

In [59]:
# # To classify based on the 'bigLetter' attribute for all ROIs
# results = decode_conditions_time_binning(dat, stimulus_conditions, classifier='hist_gradient_boosting', cv_folds=5, bin_size=0.2, classify_by='bigLetter')

# To classify based on the 'bigLetter' attribute for specific ROIs
# specific_rois = ['occ_best_filtered']
# results = decode_conditions_time_binning(dat, stimulus_conditions, classifier='hist_gradient_boosting', cv_folds=5, bin_size=0.3, classify_by='bigLetter', rois=specific_rois)

occ_decoding_results = decode_conditions_time_binning(
    dat, 
    stimulus_conditions, 
    classifier='hist_gradient_boosting', 
    cv_folds=5, 
    bin_size=0.3, 
    classify_by='bigLetter', 
    rois=['occ']
)

# # To classify based on the 'smallLetter' attribute for specific ROIs
# results = decode_conditions_time_binning(dat, stimulus_conditions, classifier='hist_gradient_boosting', cv_folds=5, bin_size=0.1, classify_by='smallLetter', rois=specific_rois)

# # To classify based on the 'task' attribute for all ROIs
# results = decode_conditions_time_binning(dat, stimulus_conditions, classifier='hist_gradient_boosting', cv_folds=5, bin_size=0.1, classify_by='task')


condition map:  {'Stimulus/BigLetters/SmallLetterh/Taskg': 's', 'Stimulus/BigLetters/SmallLetterh/Taskl': 's', 'Stimulus/BigLetters/SmallLetters/Taskg': 's', 'Stimulus/BigLetters/SmallLetters/Taskl': 's', 'Stimulus/BigLetterh/SmallLetterh/Taskg': 'h', 'Stimulus/BigLetterh/SmallLetterh/Taskl': 'h', 'Stimulus/BigLetterh/SmallLetters/Taskg': 'h', 'Stimulus/BigLetterh/SmallLetters/Taskl': 'h'}
unique labels:  ['s', 'h']
label to index:  {'s': 0, 'h': 1}
Decoding conditions for ROI: occ


KeyboardInterrupt: 

define colors for plotting (not used yet as of 8/21)

In [17]:
# Define colors for the model names
colors = {
    'congruency': 'red',
    'switchType': 'blue',
    'congruencyProportion': 'pink',
    'switchProportion': 'skyblue',
    'congruency_congruencyProportion': 'hotpink',
    'congruency_congruency_proportion': 'hotpink',
    'switchType_switchProportion': 'gray',
    'switch_type_switch_proportion': 'gray',
    'bigLetter': 'green',
    'big_letter': 'green',
    'smallLetter': 'orange',
    'small_letter': 'orange',
    'task': 'gray',
    'c75.0': 'pink',
    'i75.0': 'pink',
    'c25.0': 'gold',
    'i25.0': 'gold',
    'r25.0': 'lightblue',
    's25.0': 'lightblue',
    'r75.0': 'purple',
    's75.0': 'purple'
}

# Define linestyles for the model names
linestyles = {
    'big letter S': '-',
    'BigLetters': '-',

    'big letter H': '--',
    'BigLetterh': '--',

    'small letter S': '-',
    'SmallLetters': '-',

    'small letter H': '--',
    'SmallLetterh': '--',

    'task G': '-',
    'Taskg': '-',

    'task L': '--',
    'Taskl': '--',

    'congruent': '-',
    'c': '-',

    'incongruent': '--',
    'i': '--',

    'repeat': '-',
    'r': '-',

    'switch': '--',
    's': '--',

    'c25.0': '-',
    'c75.0': '-',
    'i25.0': '--',
    'i75.0': '--'
}